# ニューラルネットワークの動作確認

<a href="http://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn-neural-network-mlpclassifier"><b>sklearn.neural_network.MLPClassifier</b></a> を使用します。

多層パーセプトロン分類器（Multi-layer Perceptron classifier）とのことです。

テストデータ「test_daikin_conversation.csv」を使用し、動きを簡単に確認してみます。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

### (1-1) テストデータをコピー

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
    'test_benefitone_conversation.csv',
    'test_septeni_conversation.csv',
    'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


### (1-2) 訓練データのTF-IDFベクターを作成

In [3]:
X, y, vectorizer = TestTool.prepare_tf_idf_vectors(temp_path[0])

2017/04/01 PM 10:03:55 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/04/01 PM 10:03:55 TextArray#__init__ start
2017/04/01 PM 10:04:05 TextArray#to_vec start
2017/04/01 PM 10:04:06 TextArray#to_vec end


## (2) 動作確認

訓練データを使用し、学習 --> 予測 --> accuracy単純算出の流れで、動作確認をいたします。

### (2-1) 学習

In [4]:
from time import time
from sklearn.neural_network import MLPClassifier

print("MLPClassifier: fitting...")
t0 = time()

cls = MLPClassifier(activation='logistic', shuffle=False, random_state=0)
cls.fit(X, y)
print("MLPClassifier: done in %0.3fs." % (time() - t0))

MLPClassifier: fitting...
MLPClassifier: done in 168.174s.


### (2-2) 訓練データを使用して予測

In [5]:
print("MLPClassifier: predicting...")
t0 = time()

y_pred = cls.predict(X) # 予測クラスが戻る
y_proba = cls.predict_proba(X) # 予測クラスの proba が戻る
print("MLPClassifier: done in %0.3fs." % (time() - t0))

MLPClassifier: predicting...
MLPClassifier: done in 0.587s.


### (2-3) accuracy を算出

In [6]:
''' 
    正解率を計算します。
    ---> 0.99570102029118424
'''
TestTool.accuracy_score(y, y_pred, y_proba)

0.99570102029118424

## (3) 未知のデータで予測実行

### (3-1) 未知データのTF-IDFベクターを作成

明らかにおかしい質問文と、訓練データに似た質問文で、予測を実行させてみます。

In [7]:
questions = [
    '要素技術は自然languageの機械learningですか？', # まったくfeatureが抽出されない質問文

    '人生相談をしたいのですが？', # featureが１件抽出されるな質問文「する」を含む
    '難解なプログラミング技術を調達？', # featureが１件抽出される質問文「する」を含まない

    '何か習い事をしますか？', # featureが２件抽出される質問文「する」を含む
    '何か習い事がいいですか？', # featureが２件抽出される質問文「する」を含まない
    
    '何か習い事をしたほうがいいですか？', # featureが３件抽出される質問文「する」を含む
    '何か習い事を行うほうがいいですか？', # featureが３件抽出される質問文「する」を含まない
    
    '会社を辞めたいのですが誰に相談するのがいいですか？', # featureが４件抽出される質問文「する」を含む
    '会社を辞めたいのですが誰に相談を行うのがいいですか？', # featureが４件抽出される質問文「する」を含まない

    '有給休暇を取って海外に行き旅行する意向があります。', # featureが５件抽出される質問文「する」を含む
    '有給休暇を取って海外に行きたいと思っています。', # featureが５件抽出される質問文「する」を含まない

    # 以下、訓練データの質問文と似たような feature を与えてみます。
    'ウィンドウズ７の標準バージョンは？', # Windows7社内標準のバージョンを教えてください。,3637
    'ウィンドウズのパスワードを使ってログインできない', # Windowsユーザアカウントのパスワードを入力してもログインできない。,3739
    '教育コースって何？', # 教育コースについて、教えて欲しい。,3907
    '全社の規約が知りたいですが？', # 全社規程を確認したい。,3910

    # なぜか回答不能に陥った feature はこちら（正解文と一緒にテストします）
    'アウトルックのメールの探し方がわからないんですが', # Outlook2010で、メールを探すことができません。,3442
    'Outlook2010で、メールを探すことができません。', # Outlook2010で、メールを探すことができません。,3442
]
X_test = TestTool.prepare_tf_idf_vectors_from_questions(questions, vectorizer)

2017/04/01 PM 10:06:54 TextArray#__init__ start
2017/04/01 PM 10:06:54 TextArray#to_vec start
2017/04/01 PM 10:06:54 TextArray#to_vec end


### (3-2) 未知データで予測

In [8]:
print("MLPClassifier: predicting...")
t0 = time()

y_test_pred = cls.predict(X_test) # 予測クラスが戻る
y_test_proba = cls.predict_proba(X_test) # 予測クラスの proba が戻る
print("MLPClassifier: done in %0.3fs." % (time() - t0))

MLPClassifier: predicting...
MLPClassifier: done in 0.001s.


In [9]:
X_test

<17x1968 sparse matrix of type '<class 'numpy.float64'>'
	with 58 stored elements in Compressed Sparse Row format>

### (3-3) 予測結果を確認

概ね良好な結果を示しております。

In [10]:
import numpy

dumped_features = TestTool.get_dumped_features(X_test, vectorizer.vocabulary_)
for i, d in enumerate(dumped_features):
    c = y_test_pred[i]
    p = numpy.max(y_test_proba[i])
    if c == 0 or p < 0.5:
        print('%s ---> Unable to predict' % d)
    else:
        print('%s ---> class=%d (proba=%0.3f)' % (d, c, p))

index=0[] ---> Unable to predict
index=1[する=1.000] ---> Unable to predict
index=2[調達=1.000] ---> Unable to predict
index=3[する=0.707 何=0.707] ---> Unable to predict
index=4[いい=0.707 何=0.707] ---> Unable to predict
index=5[いい=0.577 する=0.577 何=0.577] ---> Unable to predict
index=6[いい=0.577 何=0.577 行う=0.577] ---> Unable to predict
index=7[いい=0.500 する=0.500 会社=0.500 誰=0.500] ---> Unable to predict
index=8[いい=0.500 会社=0.500 行う=0.500 誰=0.500] ---> Unable to predict
index=9[する=0.447 休暇=0.447 取る=0.447 海外=0.447 行く=0.447] ---> Unable to predict
index=10[休暇=0.447 取る=0.447 思う=0.447 海外=0.447 行く=0.447] ---> Unable to predict
index=11[ウィンドウズ=0.500 バージョン=0.500 標準=0.500 ７=0.500] ---> class=3637 (proba=0.989)
index=12[できる=0.408 ない=0.408 ウィンドウズ=0.408 パスワード=0.408 ログイン=0.408 使う=0.408] ---> class=3739 (proba=0.963)
index=13[コース=0.577 何=0.577 教育=0.577] ---> class=3907 (proba=0.844)
index=14[全社=0.577 知る=0.577 規約=0.577] ---> class=3910 (proba=0.978)
index=15[ない=0.378 わかる=0.378 アウト=0.378 メール=0.378 ルック=0.378 探す=0

## (4) 以下は参考です。

### (4-1) accuracy算出段階でNGとなったデータ

accuracy=99.57% となりましたが、残りの 0.43%（75件）のデータを参考までにリストします。

6件は正解にもかかわらず proba<0.5 のデータ、残りの69件は誤答とのことでした。

In [11]:
def error_detected(X, y, y_pred, y_probas):
    n_sample = X.shape[0]
    pred_err_cnt = 0
    unable_pred_cnt = 0

    y_proba = numpy.max(y_probas, axis=1)

    detected_list = []
    for i in range(n_sample):
        if y[i] == y_pred[i]:
            if y_proba[i] > 0.5:
                # しきい値を超えたもの（正しく予測できたので正答）
                pass
            else:
                # しきい値を超えなかったもの（これは回答不能）
                detected_list.append((i, X[i], y[i], y_pred[i], y_proba[i]))
                unable_pred_cnt += 1
        else:
            if y_proba[i] > 0.5:
                # しきい値を超えたもの（これは誤答）
                detected_list.append((i, X[i], y[i], y_pred[i], y_proba[i]))
                pred_err_cnt += 1
            else:
                # しきい値を超えなかったもの（正しく予測できなかったが回答不能：これはある意味正答）
                pass

    print('sample=%d, error=%d(Unable to predict=%d, Prediction error=%d)' % (
        n_sample, unable_pred_cnt+pred_err_cnt, unable_pred_cnt, pred_err_cnt))
    
    return detected_list

In [12]:
detected_list = error_detected(X, y, y_pred, y_proba)

sample=17446, error=75(Unable to predict=6, Prediction error=69)


In [13]:
'''
    上位１５件ほど表示
'''
for i, X, y, y_pred, y_proba in detected_list[0:15]:
    dump_str = TestTool.dump_features(X.toarray()[0], vectorizer.vocabulary_)
    if (y == y_pred):
        print(y, dump_str, y_pred, y_proba, 'o ---> Unable to predict')
    else:
        print(y, dump_str, y_pred, y_proba, 'x ---> Prediction error')

3642 [mcafee=0.224 する=0.447 れる=0.224 アイコ=0.224 クリック=0.224 ステータス=0.224 セキュリティ=0.224 マーク=0.224 右=0.224 対処=0.224 教える=0.224 方法=0.224 欲しい=0.224 表示=0.447] 3531 0.589123447925 x ---> Prediction error
3534 [it=0.378 ツール=0.378 何=0.378 収集=0.378 情報=0.378 管理=0.378 資産=0.378] 3957 0.604830744285 x ---> Prediction error
3536 [it=0.408 ツール=0.408 収集=0.408 情報=0.408 管理=0.408 資産=0.408] 3534 0.537643720481 x ---> Prediction error
3703 [re=0.180 spam=0.180 する=0.718 どう=0.180 なる=0.180 よい=0.180 れる=0.180 メール=0.359 件名=0.180 判別=0.180 誤る=0.180 返信=0.180 迷惑=0.180] 3948 0.500026913492 x ---> Prediction error
3938 [こんにちは=1.000] 3931 0.50193203953 x ---> Prediction error
3936 [おなか=0.707 すく=0.707] 3934 0.73912800196 x ---> Prediction error
3957 [it=0.408 ツール=0.408 収集=0.408 情報=0.408 管理=0.408 資産=0.408] 3534 0.537643720481 x ---> Prediction error
3964 [mcafee=0.229 する=0.459 れる=0.229 アイコ=0.229 クリック=0.229 ステータス=0.229 セキュリティ=0.229 マーク=0.229 右=0.229 対処=0.229 教える=0.229 方法=0.229 表示=0.459] 3531 0.504700387736 x ---> Prediction er

### (4-2) 誤答する可能性が高いデータ

上記調査結果を受け、一例として、誤答する可能性が高いデータを調査してみます。

In [14]:
'''
誤答の例：
3534 [it=0.378 ツール=0.378 何=0.378 収集=0.378 情報=0.378 管理=0.378 資産=0.378] 3957 0.604830744285 x ---> Prediction error
'''
questions2 = [
    '情報収集ツールにはどんなものがありますか？', # 正解＝3957
    'IT資産管理の情報収集ツールとは何ですか？', # 正解＝3534 <--- これは誤答する可能性が高そう・・・
]
X_test2 = TestTool.prepare_tf_idf_vectors_from_questions(questions2, vectorizer)

y_test_pred = cls.predict(X_test2) # 予測クラスが戻る
y_test_proba = cls.predict_proba(X_test2) # 予測クラスの proba が戻る

2017/04/01 PM 10:06:55 TextArray#__init__ start
2017/04/01 PM 10:06:55 TextArray#to_vec start
2017/04/01 PM 10:06:55 TextArray#to_vec end


#### 残念ながら、２番目の質問文が誤答されてしまいました。

proba は 60% と高率でした

In [15]:
y_test_pred

array([3957, 3957])

In [16]:
numpy.max(y_test_proba, axis=1)

array([ 0.98864174,  0.60483074])